In [1]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, value

# Cargar los datos del archivo CSV
data = pd.read_csv("Cheapestelectriccars-EVDatabase 2023.csv")

# Crear el problema de optimización
model = LpProblem(name="Fleet_Optimization", sense=LpMinimize)

# Crear variables binarias para cada vehículo
vehicles = list(data.index)
x = LpVariable.dicts("Vehicle", vehicles, cat="Binary")

# Definir la función objetivo: Minimizar el costo total de la flota
model += lpSum(data["PriceinGermany"][i] * x[i] for i in vehicles)

# Restricciones
model += lpSum(x[i] for i in vehicles) >= 1  # Al menos un vehículo con aceleración < 5 seg
model += lpSum(data["TopSpeed"][i] * x[i] for i in vehicles) >= 200  # Velocidad máxima de la flota > 200 km/h
model += lpSum(data["Range"][i] * x[i] for i in vehicles) >= 500  # Autonomía combinada de la flota > 500 km
model += lpSum(x[i] for i in vehicles if "All Wheel Drive" in data["Drive"][i]) >= 1  # Al menos un vehículo con tracción en las cuatro ruedas
model += lpSum(data["PriceinGermany"][i] * x[i] for i in vehicles) >= 50000  # Precio total mínimo
model += lpSum(data["PriceinGermany"][i] * x[i] for i in vehicles) <= 70000  # Precio total máximo

# Resolver el problema
model.solve()

# Mostrar la solución
selected_vehicles = [i for i in vehicles if x[i].value() == 1]
for i in selected_vehicles:
    print(f"Selected Vehicle: {data['Name'][i]}")

print(f"Total Cost: {value(model.objective)}")

TypeError: must be real number, not str